In [1]:
import pandas as pd
import numpy as np

# 
# assign the path of the code-for-nashville open data portal on githup to a variable
# 
dataPath = 'https://raw.githubusercontent.com/code-for-nashville/open-data-portal/feature/scooter-2019-08-clean-up/nashville/scooter-data/'

# 
# Make a list of all the files to download from the open data portal
# currently files for Just 22 through August 5 are available
# 
fileNames = ['scooter_extract_2019-07-'+str(x)+'.csv' for x in range(22,32)]
fileNames = fileNames + ['scooter_extract_2019-08-0'+str(x)+'.csv' for x in range(1,6)]

# 
# make a list of the columns for the facts table
# 
factColumns = ['availability_duration', 'availability_duration_seconds',
       'availability_start_date', 'availability_start_date_cst',
       'availability_start_time', 'availability_start_time_cst',
       'company_name', 'extract_date_cst',
       'extract_date_utc', 'extract_time_cst', 'extract_time_utc',
       'gps_latitude', 'gps_longitude', 'real_time_fare',
       'sumd_id']

# 
# make a list of the columns for the company dimension table and sumd dimension table
# 
companyColumns = ['company_name', 'company_phone', 'company_website']
sumdColumns = ['company_name', 'sumd_group', 'sumd_id', 'sumd_type']

In [2]:
%%time
# 
# load all the data files into a single dataframe
# this take approximately 8 minutes to load these 15 files
# 
rawData = pd.concat([pd.read_csv(dataPath+f) for f in fileNames], sort = False)

CPU times: user 52.7 s, sys: 7.54 s, total: 1min
Wall time: 4min 1s


In [3]:
%%time
# 
# create fact and dimension tables
# 
rawData['company_name'] = [x.upper() for x in rawData['company_name']]
rawData['sumd_group'] = [x.upper() for x in rawData['sumd_group']]
company = rawData[companyColumns].drop_duplicates()
sumd = rawData[sumdColumns].drop_duplicates()
sumd = sumd[sumd['sumd_group']=='SCOOTER']
scooterFacts = rawData[rawData['sumd_group']=='SCOOTER']
scooterFacts = scooterFacts[factColumns]

CPU times: user 14.8 s, sys: 2.77 s, total: 17.6 s
Wall time: 19.8 s


In [4]:
# 
# Create two new columns with the latitude and longitdue rounded to 3 places
# Using this rounded location, will allow for scooters within about 350 ft of each other
# to appear in the same location, thus minimizing the number of unique locations.
# 
scooterFacts['latitude_rnd'] = round(scooterFacts['gps_latitude'], 3)
scooterFacts['longitude_rnd'] = round(scooterFacts['gps_longitude'], 3)

In [5]:
# 
# How many scooters does each company have in Nashville?
# 
companyStats = sumd[['company_name', 'sumd_id']] \
                .groupby('company_name').count() \
                .reset_index() \
                .rename(columns={'company_name': 'Company', 'sumd_id': 'Number Of Scooters'})

companyStats

,Company,Number Of Scooters
0,BIRD,1638
1,BOLT,214
2,GOTCHA,227
3,LIME,685
4,LYFT,827
5,SPIN,977
6,UBER,1052


In [6]:
# 
# What are the 25 most popular scooters?
# The table below shows the 25 scooters that were reported in the most locations in a day.
# the numbers under the 'latitude_rnd' and 'longitude_rnd' columns represent the average number
# of locations on each day in the dataset.
# 
numOfLocsPerDay = scooterFacts[['availability_start_date_cst', 'latitude_rnd', 'longitude_rnd', 'sumd_id']] \
                    .drop_duplicates() \
                    .groupby(['sumd_id', 'availability_start_date_cst']).count() - 1

avgLocsPerDay = numOfLocsPerDay.groupby('sumd_id').mean()

totLocs = numOfLocsPerDay.groupby('sumd_id').sum()

twtyfiveMostMovedScooters = avgLocsPerDay \
                            .sort_values(by='latitude_rnd', ascending = False) \
                            .head(25) \
                            .merge(sumd[['company_name', 'sumd_id']], on='sumd_id')
twtyfiveMostMovedScooters

,sumd_id,latitude_rnd,longitude_rnd,company_name
0,PoweredLE73UC4RVRFYV,33.000000,33.000000,LIME
1,Poweredbb60ef04-c719-597f-805f-20978964fd8c,22.500000,22.500000,UBER
2,Poweredb10f56a6-f3c1-5488-9bd0-994a6f452423,18.500000,18.500000,UBER
3,Powered72839f71-6cd6-5292-8540-64e768633430,18.000000,18.000000,UBER
4,PoweredDLDCRRLYUHP5Y,17.000000,17.000000,LIME
5,Powered0a17d357-094a-57d8-8a5d-08eaa41186e0,17.000000,17.000000,UBER
6,Powered86dae5f2-dc59-509d-9f0f-d28bb7b4518c,16.600000,16.600000,UBER
7,Powered97dc7762-4b06-57d1-999d-90ffdb42ae8c,16.428571,16.428571,UBER
8,Powered85b72f09-9877-5609-a138-0a34015fb3ca,15.571429,15.571429,UBER
9,Powered0e871aa8-de0a-5b5b-9909-333de5502aa0,15.555556,15.555556,UBER


In [7]:
# 
# Calculate the total number of rides per company
# over all of the days in the dataset (15 days)
# 
companyStats = totLocs \
                .merge(sumd[['company_name', 'sumd_id']], on='sumd_id') \
                .groupby('company_name') \
                .sum() \
                .reset_index()[['company_name', 'latitude_rnd']] \
                .rename(columns={'company_name': 'Company', 'latitude_rnd': 'Total Rides'}) \
                .merge(companyStats, on='Company')

companyStats['Avg Rides Per Scooter'] = companyStats['Total Rides'] / companyStats['Number Of Scooters']

companyStats.sort_values(by=['Total Rides'], ascending = False)

,Company,Total Rides,Number Of Scooters,Avg Rides Per Scooter
0,BIRD,74114,1638,45.246642
3,LIME,38745,685,56.562044
6,UBER,23574,1052,22.408745
4,LYFT,11536,827,13.949214
5,SPIN,10364,977,10.607984
1,BOLT,1005,214,4.696262
2,GOTCHA,141,227,0.621145
